In [ ]:
!pip install langchain

In [ ]:
import logging

# Set the global logging level to WARN to reduce verbosity
logging.basicConfig(level=logging.WARN)

# This will ensure that only WARN, ERROR, and CRITICAL messages are displayed


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
import os
print(os.getenv('KAMIWAZA_DEBUG_MODE'))

In [ ]:
import ray
import ray.data
from typing import List, Dict, Any
from kamiwaza.services.retrieval.runners import FileRunner
from kamiwaza.middleware.embedding import SentenceTransformerEmbedding
from kamiwaza.middleware.vectordb import MilvusImplementation
from kamiwaza.services.catalog.services import DataService
from datahub.configuration.common import ConfigModel
from kamiwaza.services.retrieval.lib.path_spec import PathSpec

ray.init(address="auto", logging_level=logging.WARNING, log_to_driver=False)
logging.getLogger('ray').setLevel(logging.ERROR)
logging.getLogger('ray.data').setLevel(logging.ERROR)



In [ ]:
# in this null op, we just return the code, but we could do 
# any sort of preprocessing here - see wiki example
from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)
def ProcessCode(chunk: ray.data.Dataset) -> List[Dict[str, Any]]:
    from langchain.text_splitter import (
        Language,
        RecursiveCharacterTextSplitter,
    )
    # chunk has 'item' if binary, 'text' if text, 'path' unless over-ridden
    try:
        text = chunk['bytes'].decode('utf-8')
    except Exception:
        print(f"text in {chunk['path']+' ' if 'path' in chunk else ''}was not decodable")
        return []
    python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
        chunk_size=1000, chunk_overlap=0
    )

    # note splitter takes lists of texts, but strings are lists, so must enclose
    docs = python_splitter.create_documents([text])

    # we won't be using Langchain "document" objects so we will take the chunks
    # kamiwaza will automatically join them up to near the embedder limit

    chunks = [{'model_name': 'BAAI/llm-embedder', 'source': chunk['path'] if 'path' in chunk else None, 'text': doc.page_content} for doc in docs]
    return chunks
    


In [ ]:
# ingest code into catalog
catalog = DataService()
if os.path.exists('/home/matt/nvme/code/sample/kamiwaza/'):
    platform='linux'
    catalog.ingest_by_path('/home/matt/nvme/code/sample/kamiwaza/')
elif os.path.exists('/Users/matt/nvme/code/sample/kamiwaza/'):
    platform='osx'
    catalog.ingest_by_path('/Users/matt/nvme/code/sample/kamiwaza/')
else:
    print("### Error: neither sample path exissts")

In [ ]:
# Add schema to vectordb
#metadata = [
#    ('model_name', str),
#    ('source', str),
#]
vectordb = MilvusImplementation()
vectordb.drop_schema('kamiwazacode')
result = vectordb.add_schema('kamiwazacode', dims=768)

In [ ]:
run = FileRunner(catalog.get_datasets('kamiwaza'))
ds = run.get_ray_datasets(ray_kwargs={'parallelism': 24})


In [ ]:

res = ds.flat_map(ProcessCode).map_batches(
    SentenceTransformerEmbedding,
    concurrency=10 if platform == 'linux' else 4,
    num_gpus=0.1 if platform == 'linux' else 0,
    batch_size=16384,
    zero_copy_batch=True,
).map_batches(
    MilvusImplementation,
    concurrency=10 if platform == 'linux' else 4,
    num_gpus=0.1 if platform == 'linux' else 0,
    batch_size=16384,
    zero_copy_batch=True,
    fn_kwargs={'collection_name': 'kamiwazacode'}
).take_all()

print(res)


```
@ray.remote
def printpkg():
    import os
    import subprocess
    return subprocess.run('pip freeze', shell=True, check=True, capture_output=True, text=True)

res = printpkg.remote()
print(ray.get(res).stdout)

@ray.remote
def printmyenv():
    import os
    return os.getenv('VIRTUALENV')

res = printmyenv.remote()
print(ray.get(res))
```


In [ ]:
# Simple search

st = SentenceTransformerEmbedding()
query = "How do I connect to the vector database in Kamiwaza?"

# usecase and mode allow Kamiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector('kamiwazacode', query_embedding, output_fields=["*"])
vdb_search_results = search_results # save for future

for i in range(0,min(len(search_results[0]),10),1):
    print(f"{search_results[0].distances[i]}\t{search_results[0][i].entity.get('model_name')}\t{search_results[0][i].entity.get('source')}")



In [ ]:
# verbatim search search

st = SentenceTransformerEmbedding()
query = "How do I list the model inventory programmaticaly?"

# usecase and mode allow Kmiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector('kamiwazacode', query_embedding, output_fields=["*"])
modellist_search_results = search_results # save for future


for i in range(0,min(len(search_results[0]),10),1):
    print(f"{search_results[0].distances[i]}\t{search_results[0][i].entity.get('model_name')}\t{search_results[0][i].entity.get('source')}")


In [ ]:
# unrelated search search

st = SentenceTransformerEmbedding()
query = "What movie paired sandra bullock and wesley snipes early in her career?"

# NOTE: the ingest_embed_analyze_code.ipynb code actually contains this example
# so it will/should show up highly if that's the case :)

# usecase and mode allow Kmiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector('kamiwazacode', query_embedding, output_fields=["*"])
unrelated_results = search_results # save for future


for i in range(0,min(len(search_results[0]),10),1):
    print(f"{search_results[0].distances[i]}\t{search_results[0][i].entity.get('model_name')}\t{search_results[0][i].entity.get('source')}")
